# Eat Safe, Love

# Import Data into MongoDB

To import the data into MongoDB, I used the following `mongoimport` command in the terminal:

```bash
mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
pprint(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [6]:
# review a document in the establishments collection
document = db.establishments.find_one()

pprint(document)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6594fe19bc0c9ec7bb379c6d'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "name": "Penang Flavours",
    "location": {
        "type": "Point",
        "coordinates": [float("0.08384000"), float("51.49014200")]
    },
    "cuisine": "Halal",
    "rating": None,  # The restaurant hasn't been rated yet
    "address": {
        "line1": "Penang Flavours",
        "line2": "146A Plumstead Rd",
        "city": "London",
        "postcode": "SE18 7DY"
    },
    "phone": "",
    "local_authority": {
        "code": "511",
        "name": "Greenwich",
        "website": "http://www.royalgreenwich.gov.uk",
        "email": "health@royalgreenwich.gov.uk"
    },
    "scores": {
        "hygiene": "",
        "structural": "",
        "confidence_in_management": ""
    },
    "scheme_type": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "right_to_reply": "",
    "distance": 4623.9723280747176,
    "new_rating_pending": True
}


In [9]:
# Insert the new restaurant into the collection
result = db.establishments.insert_one(new_restaurant_data)

In [10]:
# Check that the new restaurant was inserted
print(f"Inserted new restaurant with id: {result.inserted_id}")

Inserted new restaurant with id: 6594fe67b2c97263dc30ba43


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
result = db.establishments.find_one(query, projection)
business_type_id = result.get("BusinessTypeID")


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
update_query = {"name": "Penang Flavours"}
update_data = {"$set": {"BusinessTypeID": business_type_id}}

In [13]:
# Confirm that the new restaurant was updated
update_result = db.establishments.update_one(update_query, update_data)
print(f"Modified {update_result.modified_count} document")

Modified 1 document


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
count_dover = db.establishments.count_documents(query)
print(f"Number of documents with LocalAuthorityName as Dover: {count_dover}")

Number of documents with LocalAuthorityName as Dover: 994


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}
delete_result = db.establishments.delete_many(query)
print(f"Deleted {delete_result.deleted_count} documents where LocalAuthorityName is Dover")

Deleted 994 documents where LocalAuthorityName is Dover


In [16]:
# Check if any remaining documents include Dover
remaining_documents_query = {"LocalAuthorityName": "Dover"}
remaining_documents_count = db.establishments.count_documents(remaining_documents_query)

if remaining_documents_count > 0:
    print(f"There are still {remaining_documents_count} documents with LocalAuthorityName as Dover.")
else:
    print("No remaining documents with LocalAuthorityName as Dover.")

No remaining documents with LocalAuthorityName as Dover.


In [17]:
# Check that other documents remain with 'find_one'
remaining_document = db.establishments.find_one()

if remaining_document:
    print("At least one remaining document:")
    pprint(remaining_document)
else:
    print("No remaining documents.")

At least one remaining document:
{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6594fe19bc0c9ec7bb379f4c'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta':

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
from decimal import Decimal

# Update the data type from String to Decimal for longitude and latitude
update_query = {"geocode.longitude": {"$type": "string"}, "geocode.latitude": {"$type": "string"}}
update_data = {"$set": {
    "geocode.longitude": {"$toDouble": "$geocode.longitude"},
    "geocode.latitude": {"$toDouble": "$geocode.latitude"}
}}

# Perform the update
update_result = db.establishments.update_many(update_query, update_data)


Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
update_query = {"RatingValue": {"$in": non_ratings}}
update_data = {"$set": {"RatingValue": None}}


In [20]:
# Change the data type from String to Integer for RatingValue
update_query = {"RatingValue": {"$type": "string", "$exists": True}}
update_data = {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}

# Perform the update
update_result = db.establishments.update_many(update_query, update_data)


In [21]:
# Check that the coordinates and rating value are now numbers
sample_document = db.establishments.find_one()

if sample_document:
    print("Sample Document:")
    pprint(sample_document)
else:
    print("No documents found.")


Sample Document:
{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6594fe19bc0c9ec7bb379f4c'),
 'geocode': {'latitude': {'$toDouble': '$geocode.latitude'},
             'longitude': {'$toDouble': '$geocode.longitude'}},
 'links': [{'href': 'https://api.ratings.food.